<a href="https://colab.research.google.com/github/yuukienomoto/mahjong/blob/main/mj_lightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# これはコードとして書式設定されます
```

麻雀研究　lightGBMを用いた捨て牌の危険度
推定

In [ ]:
import gzip
import xml.etree.ElementTree as ET
import glob
from google.colab import drive
import time
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random

print("--- ステップ1: Google Driveのマウント ---")
try:
    drive.mount('/content/drive')
except:
    pass

# 2. フォルダパス指定
log_folder_path = '/content/drive/MyDrive/mjlog_pf4-20_n28/'
file_pattern = log_folder_path + '*.mjlog'
log_files = glob.glob(file_pattern)

if not log_files:
    print(f"エラー: {log_folder_path} に .mjlog ファイルが見つかりません。")
else:
    print(f"全 {len(log_files)} 個のファイルが見つかりました。")

    # ★★★ 最終的なデータセット ★★★
    final_dataset = []

    # ★★★【最重要】本格的な特徴量計算関数 ★★★
    #  に基づく
    def calculate_features(game_state, player_index, tile):

        features = []

        # 1. 捨てようとしている牌の種類 [cite: 31]
        # (牌コードをそのまま特徴量とする)
        features.append(tile)

        # 2. 捨てようとしている牌が場に見えている枚数 [cite: 33]
        visible_count = 0
        for hand in game_state['hands']:
            visible_count += hand.count(tile)
        for discard_pile in game_state['discards']:
            visible_count += discard_pile.count(tile)
        features.append(visible_count)

        # 3. 相手のリーチの有無 [cite: 34]
        opponent_reach = False
        for i in range(4):
            if i != player_index and game_state['reach'][i]:
                opponent_reach = True
        features.append(1 if opponent_reach else 0)

        # 4. 捨てようとしている牌が相手の河に存在するか [cite: 36]
        in_opponent_discard = False
        for i in range(4):
            if i != player_index and tile in game_state['discards'][i]:
                in_opponent_discard = True
        features.append(1 if in_opponent_discard else 0)

        # 5. 1巡以内 (4打) に誰かが捨てているか [cite: 37]
        discarded_1_turn = False
        # 直近4回の打牌履歴 (自分も含む) をチェック
        recent_discards = game_state['history'][-4:]
        for (p, t) in recent_discards:
            if t == tile:
                discarded_1_turn = True
        features.append(1 if discarded_1_turn else 0)

        # 6. 2巡以内 (8打) に誰かが捨てているか [cite: 38]
        discarded_2_turns = False
        # 直近8回の打牌履歴 (自分も含む) をチェック
        recent_discards = game_state['history'][-8:]
        for (p, t) in recent_discards:
            if t == tile:
                discarded_2_turns = True
        features.append(1 if discarded_2_turns else 0)

        # (ここで特徴量 [cite: 32, 35] を追加実装できる)
  # 牌の種類と番号を計算するヘルパー関数
        def get_tile_info(tile_code):
            # 0-33: 数牌 (0-8:m, 9-17:p, 18-26:s), 27-33: 字牌
            if tile_code < 27:
                suit = tile_code // 9  # 0=m, 1=p, 2=s
                number = (tile_code % 9) + 1 # 1-9
                return suit, number
            return None, None # 字牌は順子にならない

        # ★★★ ここから追加した特徴量 ★★★
        # 7. 捨てようとしている牌と手牌で順子が作れるか
        can_form_shuntsu = 0
        temp_hand = game_state['hands'][player_index] + [tile]
        tile_counts = {}
        for t in temp_hand:
            tile_counts[t] = tile_counts.get(t, 0) + 1

        target_suit, target_number = get_tile_info(tile)

        if target_suit is not None:
            # パターン1: (N-2, N-1, N)
            if target_number >= 3:
                tile_n_minus_2 = target_suit * 9 + (target_number - 3)
                tile_n_minus_1 = target_suit * 9 + (target_number - 2)
                if tile_counts.get(tile_n_minus_2, 0) >= 1 and \
                   tile_counts.get(tile_n_minus_1, 0) >= 1:
                    can_form_shuntsu = 1

            # パターン2: (N-1, N, N+1)
            if target_number >= 2 and target_number <= 8:
                tile_n_minus_1 = target_suit * 9 + (target_number - 2)
                tile_n_plus_1 = target_suit * 9 + target_number
                if tile_counts.get(tile_n_minus_1, 0) >= 1 and \
                   tile_counts.get(tile_n_plus_1, 0) >= 1:
                    can_form_shuntsu = 1

            # パターン3: (N, N+1, N+2)
            if target_number <= 7:
                tile_n_plus_1 = target_suit * 9 + target_number
                tile_n_plus_2 = target_suit * 9 + target_number + 1
                if tile_counts.get(tile_n_plus_1, 0) >= 1 and \
                   tile_counts.get(tile_n_plus_2, 0) >= 1:
                    can_form_shuntsu = 1

        features.append(can_form_shuntsu)

        # 8. スジ牌, 9. カベ牌

        # 場に見えている全ての牌のリストを作成 (手牌、河、鳴き牌)
        all_visible_tiles = []
        for i in range(4):
            # 他家の河牌のみをチェック対象とする (自分の手牌は関係なし)
            if i != player_index:
                all_visible_tiles.extend(game_state['discards'][i])
            # 自分の手牌はカウントしないが、鳴き牌 (ここではdiscardsとhandsの合計で代用) は考慮する
            # ※ mjlogの構造上、鳴き牌はdiscardsに含まれるため、discardsのみでチェック可能

        # 8. 捨てようとしている牌がスジ牌であるか
        # リーチ者の捨て牌に対して、そのスジ牌であるかをチェックする
        is_suji_tile = 0

        if target_suit is not None and target_number in {1, 2, 3, 4, 5, 6, 7, 8, 9}:
            suji_pairs = {
                1: [4, 7], 2: [5, 8], 3: [6, 9],
                4: [1, 7], 5: [2, 8], 6: [3, 9],
                7: [1, 4], 8: [2, 5], 9: [3, 6]
            }

            # 捨て牌がスジ牌の中心（例：5）に対するスジ（例：2, 8）かどうかをチェック
            target_sujis = suji_pairs.get(target_number, [])

            for i in range(4):
                if i != player_index and game_state['reach'][i]: # リーチしている他家に対して
                    opponent_discards = game_state['discards'][i]

                    for discard_tile in opponent_discards:
                        disc_suit, disc_number = get_tile_info(discard_tile)

                        if disc_suit == target_suit: # 同じ種類
                            # 捨て牌がスジの関係にある牌か (例: 1を捨てているなら4, 7がスジ)

                            # 捨て牌が3の倍数+1 (1, 4, 7)
                            if disc_number in {1, 4, 7}:
                                if target_number == disc_number + 3 or target_number == disc_number + 6:
                                    is_suji_tile = 1
                                    break
                            # 捨て牌が3の倍数+2 (2, 5, 8)
                            elif disc_number in {2, 5, 8}:
                                if target_number == disc_number + 3 or target_number == disc_number - 3:
                                    is_suji_tile = 1
                                    break
                            # 捨て牌が3の倍数 (3, 6, 9)
                            elif disc_number in {3, 6, 9}:
                                if target_number == disc_number - 3 or target_number == disc_number - 6:
                                    is_suji_tile = 1
                                    break

                    if is_suji_tile == 1:
                        break

        features.append(is_suji_tile)

        # 9. 捨てようとしている牌の隣接牌がカベになっているか
        # (N-2)が4枚見えている: (N-1)(N)待ちなし
        # (N-1)が4枚見えている: (N)(N+1)待ちなし
        # (N+1)が4枚見えている: (N-1)(N)待ちなし
        # (N+2)が4枚見えている: (N)(N-1)待ちなし

        is_kabe_tile = 0

        if target_suit is not None:
            # 1. N-1がカベになっているか (N, N+1 のリャンメン待ちの危険度を下げる)
            if target_number >= 2:
                tile_n_minus_1_code = target_suit * 9 + (target_number - 2)
                if all_visible_tiles.count(tile_n_minus_1_code) == 4:
                    is_kabe_tile = 1

            # 2. N+1がカベになっているか (N-1, N のリャンメン待ちの危険度を下げる)
            if target_number <= 8:
                tile_n_plus_1_code = target_suit * 9 + target_number
                if all_visible_tiles.count(tile_n_plus_1_code) == 4:
                    is_kabe_tile = 1

        features.append(is_kabe_tile)
        return features

--- ステップ1: Google Driveのマウント ---
Mounted at /content/drive
全 3181 個のファイルが見つかりました。


In [ ]:

    # --- ステップ2: 全ファイル (100個) の処理 ---
    print("\n--- ステップ2: 全ファイルの解析を開始します ---")
    start_time = time.time()

    # (テストのため、まずは最初の100ファイルだけ処理)
    files_to_process = log_files

    for i, file_path in enumerate(files_to_process):

        if (i + 1) % 20 == 0: # 20ファイルごとに進捗を表示
            print(f"  処理中... {i+1} / {len(files_to_process)} ファイル")

        try:
            with gzip.open(file_path, 'rt', encoding='utf-8') as f:
                tree = ET.parse(f)
                root = tree.getroot()

            # --- ゲーム状態変数 (Game State) ---
            game_state = {
                'hands': [[], [], [], []],
                'discards': [[], [], [], []],
                'reach': [False, False, False, False],
                'history': [] # (player, tile) の打牌履歴
            }
            last_discard_event = None

            for tag in root:
                if tag.tag == 'INIT':
                    if last_discard_event:
                        final_dataset.append(last_discard_event)

                    game_state = {
                        'hands': [[], [], [], []],
                        'discards': [[], [], [], []],
                        'reach': [False, False, False, False],
                        'history': []
                    }
                    last_discard_event = None

                    hai_strings = [tag.attrib.get(h) for h in ['hai0', 'hai1', 'hai2', 'hai3']]
                    for p_idx in range(4):
                        if hai_strings[p_idx]:
                            game_state['hands'][p_idx] = [int(s) for s in hai_strings[p_idx].split(',')]

                # ツモ
                draw_prefix = tag.tag[0]
                if draw_prefix in {'T': 0, 'U': 1, 'V': 2, 'W': 3}:
                    try:
                        player_index = {'T': 0, 'U': 1, 'V': 2, 'W': 3}[draw_prefix]
                        tile_code = int(tag.tag[1:])
                        game_state['hands'][player_index].append(tile_code)
                    except (ValueError, KeyError):
                        pass

                # 打牌
                discard_prefix = tag.tag[0]
                if discard_prefix in {'D': 0, 'E': 1, 'F': 2, 'G': 3}:
                    try:
                        if last_discard_event:
                            final_dataset.append(last_discard_event)

                        player_index = {'D': 0, 'E': 1, 'F': 2, 'G': 3}[discard_prefix]
                        tile_code = int(tag.tag[1:])

                        # ★★★ 強化された特徴量でXを計算 ★★★
                        features_X = calculate_features(
                            game_state, player_index, tile_code
                        )
                        last_discard_event = {'X': features_X, 'y': 0}

                        # ゲーム状態の更新
                        if tile_code in game_state['hands'][player_index]:
                            game_state['hands'][player_index].remove(tile_code)
                        game_state['discards'][player_index].append(tile_code)
                        game_state['history'].append((player_index, tile_code)) # 打牌履歴を記録

                    except (ValueError, KeyError):
                        pass

                # リーチ
                if tag.tag == 'REACH':
                    if tag.attrib.get('step') == '1':
                         player_index = int(tag.attrib.get('who'))
                         game_state['reach'][player_index] = True

                # 和了 (ロン)
                if tag.tag == 'AGARI':
                    winner = int(tag.attrib.get('who'))
                    loser = tag.attrib.get('fromWho')

                    if loser and winner != int(loser):
                        score = int(tag.attrib.get('ten').split(',')[1])

                        if last_discard_event:
                            last_discard_event['y'] = score
                            final_dataset.append(last_discard_event)
                            last_discard_event = None

        except Exception as e:
            print(f"  ファイル {file_path} の処理中にエラー: {e}")

    # --- ステップ2 完了 ---
    end_time = time.time()
    print("\n-------------------------")
    print(f"全 {len(files_to_process)} ファイルの解析が完了しました。")
    print(f"処理時間: {end_time - start_time:.2f} 秒")
    print(f"作成された総データセット件数: {len(final_dataset)}")





--- ステップ2: 全ファイルの解析を開始します ---
  処理中... 20 / 3181 ファイル
  処理中... 40 / 3181 ファイル
  処理中... 60 / 3181 ファイル
  処理中... 80 / 3181 ファイル
  処理中... 100 / 3181 ファイル
  処理中... 120 / 3181 ファイル
  処理中... 140 / 3181 ファイル
  処理中... 160 / 3181 ファイル
  処理中... 180 / 3181 ファイル
  処理中... 200 / 3181 ファイル
  処理中... 220 / 3181 ファイル
  処理中... 240 / 3181 ファイル
  処理中... 260 / 3181 ファイル
  処理中... 280 / 3181 ファイル
  処理中... 300 / 3181 ファイル
  処理中... 320 / 3181 ファイル
  処理中... 340 / 3181 ファイル
  処理中... 360 / 3181 ファイル
  処理中... 380 / 3181 ファイル
  処理中... 400 / 3181 ファイル
  処理中... 420 / 3181 ファイル
  処理中... 440 / 3181 ファイル
  処理中... 460 / 3181 ファイル
  処理中... 480 / 3181 ファイル
  処理中... 500 / 3181 ファイル
  処理中... 520 / 3181 ファイル
  処理中... 540 / 3181 ファイル
  処理中... 560 / 3181 ファイル
  処理中... 580 / 3181 ファイル
  処理中... 600 / 3181 ファイル
  処理中... 620 / 3181 ファイル
  処理中... 640 / 3181 ファイル
  処理中... 660 / 3181 ファイル
  処理中... 680 / 3181 ファイル
  処理中... 700 / 3181 ファイル
  処理中... 720 / 3181 ファイル
  処理中... 740 / 3181 ファイル
  処理中... 760 / 3181 ファイル
  処理中... 780 / 3181 ファイ

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random
import time

print("\n--- ステップ3: LightGBM モデルへの切り替え ---")

# (ステップ2で 'final_dataset' (136万件) が作成されていることが前提)

if (not 'final_dataset') or (len(final_dataset) == 0):
    print("エラー: 'final_dataset' が見つかりません。")
    print("「ステップ1」と「ステップ2」（解析コード）を先に実行してください。")
else:
    # 1. 136万件の全データを X と y に分離 (サンプリングは不要)
    print(f"全 {len(final_dataset)} 件のデータセットを X と y に分離しています...")
    # (特徴量は9次元のはずです)
    X = np.array([item['X'] for item in final_dataset])
    y = np.array([item['y'] for item in final_dataset])

    print(f"\n特徴量Xの形状 (全データ): {X.shape}")
    print(f"教師値yの形状 (全データ): {y.shape}")

    # 2. 訓練データとテストデータに分割 (スケーリングは不要)
    # (80%を訓練用、20%をテスト用に使います)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"訓練データ数: {len(X_train)}, テストデータ数: {len(X_test)}")

    # 3. LightGBM モデルの定義
    print("\nLightGBMモデルを定義します...")
    # SVRの C や gamma のチューニングは不要です
    lgbm_model = lgb.LGBMRegressor(
        objective='regression_l2',  # 目的 = 回帰 (平均二乗誤差)
        is_unbalance=True,          # ★不均衡データ(1% vs 99%)を自動で重み付け
        n_estimators=200,           # 木の数 (多いほど高精度だが時間がかかる)
        learning_rate=0.1,          # 学習率
        n_jobs=-1                   # Colabの全CPUを使って高速化
    )

    # 4. モデルの学習
    print("\nLightGBMモデルの学習を開始します... (SVRより遥かに高速です)")
    start_train_time = time.time()

    # eval_setでテストデータを指定し、早期終了(early_stopping)を使う
    lgbm_model.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)], # テストデータで性能を監視
        eval_metric='rmse',         # 評価指標 = RMSE (予測ズレの平均)
        callbacks=[lgb.early_stopping(10)] # 10回連続で性能が改善しなければ停止
    )

    end_train_time = time.time()
    print(f"学習が完了しました！ (学習時間: {end_train_time - start_train_time:.2f} 秒)")

    # 5. モデルの「全体的な精度」を評価
    print("\n--- モデル全体の精度評価 (テストデータ) ---")
    y_pred = lgbm_model.predict(X_test)

    # RMSE（予測ズレが平均何点か）を計算
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"モデルの予測ズレ (RMSE): {rmse:.2f} 点")
    print(f"  (参考: SVRのズレは約 3700.44 点でした)")

    # 6. 予測テスト (ランダムな10件)
    print("\n--- 予測テスト (LGBMモデル) ---")
    # テストデータからランダムに10件選ぶ
    test_indices = random.sample(range(len(X_test)), 10)

    test_X = X_test[test_indices]
    test_y_actual = y_test[test_indices]

    # 最適化されたモデルで予測
    test_y_pred = lgbm_model.predict(test_X)

    for i in range(10):
        print(f"--- サンプル{i} ---")
        print(f"  特徴量(X): {test_X[i]}")
        print(f"  実際の点数(y): {test_y_actual[i]}")
        print(f"  LGBMが予測した危険度: {test_y_pred[i]:.2f} 点")


--- ステップ3: LightGBM モデルへの切り替え ---
全 1365231 件のデータセットを X と y に分離しています...

特徴量Xの形状 (全データ): (1365231, 9)
教師値yの形状 (全データ): (1365231,)
訓練データ数: 1092184, テストデータ数: 273047

LightGBMモデルを定義します...

LightGBMモデルの学習を開始します... (SVRより遥かに高速です)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.046512 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 142
[LightGBM] [Info] Number of data points in the train set: 1092184, number of used features: 4
[LightGBM] [Info] Start training from score 53.288732
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	valid_0's rmse: 637.253	valid_0's l2: 406091
学習が完了しました！ (学習時間: 2.44 秒)

--- モデル全体の精度評価 (テストデータ) ---
モデルの予測ズレ (RMSE): 637.25 点
  (参考: SVRのズレは約 3700.44 点でした)

--- 予測テスト (LGBMモデル) ---
--- サンプル0 ---
  特徴量(X): [131   1   1   0   0   0   0   0   0]
  実際の点数(y): 0
  LGBMが予測した危険度: 57.95 点
--- サンプル1 ---
  特徴量(X): [115   1   0   0   0   0  

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd

try:
    # 1. 私たちが定義した「特徴量の名前」のリスト
    feature_names = [
        '1. 牌コード (Tile Code)',
        '2. 見え枚数 (Visible Count)',
        '3. 相手リーチ (Opponent Reach)',
        '4. 相手の河にあるか (In Opponent Discard)',
        '5. 1巡以内 (Discarded 1-Turn)',
        '6. 2巡以内 (Discarded 2-Turns)',
        '7. 順子 (Shuntsu) [新]',
        '8. スジ (Suji) [新]',
        '9. カベ (Kabe) [新]'
    ]

    # 2. LGBMモデルから「重要度」のスコアを取得
    importances = lgbm_model.feature_importances_

    # 3. データフレームを作成
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance Score': importances
    })

    # 4. 重要度が高い順に並べ替え
    feature_importance_df = feature_importance_df.sort_values(by='Importance Score', ascending=False)

    print(f"--- LightGBM 特徴量の重要度 (全 9次元) ---")

    # 5. グラフの代わりに、テキスト（数字）で結果を表示
    # .to_string() ですべての行を省略せずに表示します
    print(feature_importance_df.to_string())

except NameError as e:
    print(f"\nエラー: 必要な変数 ('lgbm_model'など) が見つかりません。 {e}")
    print("\n!! 解決策: !!")
    print("1. 「ステップ3」（LightGBMの学習）のセルを再実行してください。")
    print("2. その後、もう一度このセルを実行してください。")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")

--- LightGBM 特徴量の重要度 (全 9次元) ---
                             Feature  Importance Score
0                1. 牌コード (Tile Code)               738
2          3. 相手リーチ (Opponent Reach)                27
6                7. 順子 (Shuntsu) [新]                23
7                   8. スジ (Suji) [新]                22
1            2. 見え枚数 (Visible Count)                 0
4         5. 1巡以内 (Discarded 1-Turn)                 0
3  4. 相手の河にあるか (In Opponent Discard)                 0
5        6. 2巡以内 (Discarded 2-Turns)                 0
8                   9. カベ (Kabe) [新]                 0
